In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

# converte a velocidade de m/s para km/h
def ms2kh(ms):
  return(3.6 * ms)

# calcula o pace médio
def kh2Pace(kmh):
  hkm = 60 * 1.0 / kmh
  
  mins = int(hkm)
  segs = int((hkm - mins) * 60)
  
  return(mins, segs)

# imprime uma string s + um número f com duas casas de precisão
def myPrintFloat(s, f):
  print(s, "{:.2f}".format(f))

# estima as posições e as velocidades pelo método de euler em diversos
# instantes de tempo dada uma aceleração constante
def euler(t0, tf, dt, x0, v0, aceleracaoMedia):
  
  # inicializa com os valores iniciais  
  vatual = v0
  xatual = x0
  tatual = t0

  # vetores em que serão armazenados os valores encontrados
  v = []
  x = []
  t = []

  while(tatual <= tf):
    # adiciona os últimos valores encontrados de velocidade posição e tempo
    v.append(vatual)
    x.append(xatual)
    t.append(tatual)
    
    # calcula os valores atuais de posição velocidade e tempo
    xatual = xatual + (vatual * dt)
    vatual = vatual + (aceleracaoMedia * dt)
    tatual += dt

  return(t, x, v)

# gera os gráficos para um arquivo específico  
def graficos(nome_arquivo, i, t0_caminhada, tf_caminhada, t0_corrida, tf_corrida):
  
  # lê os dados obtidos
  dados_acelerometro = np.array(pd.read_csv(nome_arquivo, sep = ";"))

  # ------- gráfico com os dados de toda a medição -------

  tempo = dados_acelerometro[:, 0]
  aceleracao = dados_acelerometro[:, 4]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.title(f"Dados do acelerômetro - Experimento {i + 1}")
  ax.set_xlabel("Tempo (s)")
  ax.set_ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- gráfico com somente os dados do trajeto (caminhada + corrida) -------

  tempo = dados_acelerometro[t0_caminhada:tf_corrida, 0]
  aceleracao = dados_acelerometro[t0_caminhada:tf_corrida, 4]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.title(f"Dados do acelerômetro: zoom no trajeto completo - Experimento {i + 1}")
  ax.set_xlabel("Tempo (s)")
  ax.set_ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- gráficos com somente os dados da caminhada (MRU) -------

  # gera o gráfico dos dados obtidos pelo acelerometro
  tempo = dados_acelerometro[t0_caminhada:tf_caminhada, 0]
  aceleracao = dados_acelerometro[t0_caminhada:tf_caminhada, 4]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.title(f"Dados do acelerômetro: zoom na caminhada - Experimento {i + 1}")
  ax.set_xlabel("Tempo (s)")
  ax.set_ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()
  
  # gera o gráfico dos dados obtidos pelo acelerometro marcando os passos
  peaks, _ = find_peaks(aceleracao, distance = 30, prominence = 0.6)
  # ignora o último
  peaks = peaks[0:-1]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.plot(tempo[peaks], aceleracao[peaks], "x", label = "Passos")
  plt.title(f"Passos na caminhada - Experimento {i + 1}")
  plt.xlabel("Tempo (s)")
  plt.ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- obter dados da caminhada -------

  primeiroPasso = peaks[0]
  ultimoPasso = peaks[-1]

  # acelerações do intervalo que caminhou

  caminhada = np.asarray(aceleracao[primeiroPasso:ultimoPasso])

  # tempos do intervalo que caminhou
  caminhadaTempo = np.asarray(tempo[primeiroPasso:ultimoPasso])
  caminhadaTempo -= caminhadaTempo[0]

  peaks, _ = find_peaks(caminhada, distance = 30, prominence = 0.6)

  # tamanho passo
  tamanhoPasso = 0.9

  # quantidade de passos
  nPassosCaminhada = len(peaks)

  # calcular quantidade de passos e distância total
  distanciaTotal = nPassosCaminhada * tamanhoPasso

  # calcular tempo médio de cada passo e tempo total
  tempoPasso = np.average(np.diff(caminhadaTempo[peaks]))
  tempoTotal = nPassosCaminhada * tempoPasso

  # cálculo da velocidade média (usando v = d / t)
  velocidadeMedia = (distanciaTotal) / (tempoTotal)
  
  ind = np.arange(0, nPassosCaminhada) 
  
  # posições dos passos
  x = tamanhoPasso * ind
  # tempo dos passos
  t = tempoPasso * ind
  
  # posição inicial
  x0 = x[0]
  # tempo inicial
  t0 = t[0]
  # tempo final
  tf = t[-1]

  # obtém as posições estimadas pelo método de euler
  # obs: no MRU, a acelaração média é 0
  tEuler, xEuler, _ = euler(t0, tf, tempoPasso, x0, velocidadeMedia, 0)

  # salva dados obtidos
  distCaminhada = xEuler[-1] - xEuler[0]
  tempoCaminhada = tEuler[-1] - tEuler[0]
  velocidadeMediaCaminhada = distCaminhada / tempoCaminhada
  paceMinsCaminhada, paceSegsCaminhada = kh2Pace(ms2kh(velocidadeMediaCaminhada))

  # gera o gráfico da posição estimada pelo método de euler
  fig, ax = plt.subplots()
  plt.plot(tEuler, xEuler, "ro", label = "Estimado: Euler")
  plt.title(f"Posição na caminhada - Experimento {i + 1}")
  plt.xlabel("Tempo (s)")
  plt.ylabel("Posição estimada (m)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- gráficos com somente os dados da corrida (MRUV) -------

  # gera o gráfico dos dados obtidos pelo acelerômetro
  tempo = dados_acelerometro[t0_corrida:tf_corrida, 0]
  aceleracao = dados_acelerometro[t0_corrida:tf_corrida, 4]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.title(f"Dados do acelerômetro: zoom na corrida - Experimento {i + 1}")
  ax.set_xlabel("Tempo (s)")
  ax.set_ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # gera o gráfico dos dados obtidos pelo acelerômetro marcando os passos
  peaks, _ = find_peaks(aceleracao, distance = 25, prominence = 0.5)
  # ignora o último pico
  peaks = peaks[0:-1]

  fig, ax = plt.subplots()
  plt.plot(tempo, aceleracao, label = "Dados do acelerômetro")
  plt.plot(tempo[peaks], aceleracao[peaks], "x", label = "Passos")
  plt.title(f"Passos na corrida - Experimento {i + 1}")
  plt.xlabel("Tempo (s)")
  plt.ylabel("Resultante (força g)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- obter dados da corrida -------

  primeiroPasso = peaks[0]
  ultimoPasso = peaks[-1]

  # acelerações do intervalo que correu
  caminhada = np.asarray(aceleracao[primeiroPasso:ultimoPasso])

  # tempos do intervalo que caminhou
  caminhadaTempo = np.asarray(tempo[primeiroPasso:ultimoPasso])
  caminhadaTempo -= caminhadaTempo[0]
  
  peaks, _ = find_peaks(caminhada, distance = 25, prominence = 0.5)

  # tamanho passo
  tamanhoPasso = 0.9

  # quantidade de passos
  nPassosCorrida = len(peaks)

  # calcular quantidade de passos e distância total
  distanciaTotal = nPassosCorrida * tamanhoPasso

  # calcular tempo médio de cada passo e tempo total
  tempoPasso = np.average(np.diff(caminhadaTempo[peaks]))
  tempoTotal = nPassosCorrida * tempoPasso

  # calcular aceleração média (usando d = v0 * t + a * t ** 2 / 2)
  # obs: a velocidade inicial é igual à velocidade média da caminhada
  aceleracaoMedia = 2  * (distanciaTotal - velocidadeMedia * tempoTotal) / tempoTotal ** 2

  ind = np.arange(0, nPassosCorrida) 

  # posições dos passos
  x = tamanhoPasso * ind
  # tempo dos passos
  t = tempoPasso * ind

  # posição inicial
  x0 = x[0]
  # tempo inicial
  t0 = t[0]
  # tempo final
  tf = t[-1]

  # obtém as posições e as velocidades estimadas pelo método de euler
  tEuler, xEuler, vEuler = euler(t0, tf, tempoPasso, x0, velocidadeMedia, aceleracaoMedia)

  # salva dados obtidos
  tempoCorrida = tEuler[-1] - tEuler[0]
  distCorrida = xEuler[-1] - xEuler[0]
  velocidadeMediaCorrida = distCorrida / tempoCorrida
  paceMinsCorrida, paceSegsCorrida = kh2Pace(ms2kh(velocidadeMediaCorrida))
  
  # gera o gráfico da velocidade estimada pelo método de euler
  fig, ax = plt.subplots()
  plt.plot(tEuler, vEuler, "ro", label = "Estimado: Euler")
  plt.title(f"Velocidade na corrida - Experimento {i + 1}")
  plt.xlabel("Tempo (s)")
  plt.ylabel("Velocidade estimada (m/s)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # gera o gráfico da posição estimada pelo método de euler
  fig, ax = plt.subplots()
  plt.plot(tEuler, xEuler, "ro", label = "Estimado: Euler")
  plt.title(f"Posição na corrida - Experimento {i + 1}")
  plt.xlabel("Tempo (s)")
  plt.ylabel("Posição estimada (m)")
  ax = plt.gca()
  ax.legend()
  plt.show()
  print()

  # ------- dados gerais obtidos ------

  velocidadeMediaTrajeto = (distCorrida + distCaminhada) / (tempoCorrida + tempoCaminhada) 

  print("---------------------------------------------------------------------")
  print(f"Sumário de dados do experimento {i + 1}\n")

  myPrintFloat("Número de passos:", nPassosCaminhada + nPassosCorrida)
  
  myPrintFloat("Distância percorrida (m):", distCaminhada + distCorrida)
  myPrintFloat("Tempo transcorrido (s):", tempoCaminhada + tempoCorrida)
  
  myPrintFloat("Velocidade média na caminhada (m/s):", velocidadeMediaCaminhada)
  myPrintFloat("Velocidade média na corrida (m/s):", velocidadeMediaCorrida)
  myPrintFloat("Velocidade média no trajeto (m/s):", velocidadeMediaTrajeto)
  
  paceString = "Pace médio na caminhada (min/km): " + str(paceMinsCaminhada) + " min " + str(paceSegsCaminhada) + " s por km"
  print(paceString)
  paceString = "Pace médio na corrida (min/km): " + str(paceMinsCorrida) + " min " + str(paceSegsCorrida) + " s por km"
  print(paceString)

  print("---------------------------------------------------------------------\n")

def main():
  t0_caminhada = [3000, 4000, 4150, 2450, 11000]
  tf_caminhada = [4300, 5250, 5530, 3950, 12550]
  t0_corrida = [4300, 5250, 5530, 3950, 12550]
  tf_corrida = [5150, 6050, 6450, 4800, 13350]

  for i in range(5):
    graficos(f"dados_corrida{i + 1}.csv", i, t0_caminhada[i], tf_caminhada[i], t0_corrida[i], tf_corrida[i])

main()

Output hidden; open in https://colab.research.google.com to view.